In [ ]:
import pandas as pd

# Path to the CSV file
file_path = 'subset_train.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

In [ ]:
df.shape

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
import pandas as pd

new_df = df #SLICE THE DATAFRAME HERE
model_labels = []

endpoint = "" #ADD YOUR ENDPOINT HERE
deployment_name = "gpt-5-mini"
api_key = "" #ADD YOUR API KEY HERE

client = OpenAI(
    base_url=endpoint,
    api_key=api_key
)

for iter,row in new_df.iterrows():
    context = row["context"]
    question = row["question"]
    answer = row["answer"]
    sys_prompt = ""

    if context == "" or pd.isna(context):
      sys_prompt = """You are a classification model.

Your task is to assign exactly ONE label to an answer given a question.

Labels:
- \"factual\": The answer addresses the question and is accurate.
- \"contradiction\": The answer addresses the question but is incorrect.
- \"irrelevant\": The answer is not relevant to the question.

Rules:
- Output only one of the three labels.
- Do not explain your reasoning.

Decision procedure:
1. If the answer does not address the question, label \"irrelevant\".
2. If the answer addresses the question and is correct, label \"factual\".
3. If the answer addresses the question and is incorrect, label \"contradiction\"."""

    else:
      context = "Context: "+context+"\n\n"
      sys_prompt = """You are a classification model.

Your task is to assign exactly ONE label to an answer given a context and a question.

Labels:
- \"factual\": The answer addresses the question and is supported by the context.
- \"contradiction\": The answer addresses the question but is incorrect or contradicts the context.
- \"irrelevant\": The answer is not relevant to the question.

Rules:
- Use only the information provided in the context.
- Do not use outside knowledge.
- Output only one of the three labels.
- Do not explain your reasoning.

Decision procedure:
1. If the answer does not address the question, label \"irrelevant\".
2. If the answer addresses the question and is supported by the context, label \"factual\".
3. If the answer addresses the question but is contradicted by the context, label \"contradiction\"."""

    try:
      completion = client.chat.completions.create(
        model=deployment_name,
        messages=[
          {
            "role": "system",
            "content": f"{sys_prompt}"
          },
          {
            "role": "user",
            "content": f"{context}Question: {question}\n\nAnswer: {answer}\n\nLabel:",
          },
        ],
      )

      model_labels.append(completion.choices[0].message.content)
    except:
      model_labels.append("ERROR OCCURRED PLEASE FIX")

    if iter % 10 == 0:    print(iter, " done")


new_df["model_label"] = model_labels
new_df.to_csv("gpt-labelled.csv", index=False)
